# Introduction

1. Create an account on SynBioHub
2. Make sure you've downloaded `parts.xml` and it is placed somewhere convenient on your computer.
3. Make sure you've downloaded `results.txt` and it is placed somewhere convenient on your computer.
4. Install SBOL library in language of choice

# Getting a Device from an SBOL Compliant XML

In [ ]:
from sbol import *

# Set the default namespace (e.g. “http://my_namespace.org”)
namespace = "http://my_namespace.org"
setHomespace(namespace)

# Create a new SBOL document
doc = Document()

# Load some generic parts from `parts.xml` into another Document
generic_parts = Document('parts.xml')

# Copy the parts from `parts.xml` into your document. 
# Be sure to specify the original namespace `http://examples.org`
generic_parts.copy("http://examples.org", doc)

In [ ]:
for obj in doc:
    print(obj)

# Getting a Device from Synbiohub

In [ ]:
# Start an interface to igem’s public part shop on SynBioHub. Located at `https://synbiohub.org/public/igem`
partshop = PartShop('https://synbiohub.org/public/igem')

# Search the part shop for parts from the iGEM interlab study using the search term `interlab`
records = partshop.search('interlab')

# Import the medium strength device into your document
medium_device_uri = records[0].identity
partshop.pull(medium_device_uri, doc)

In [ ]:
for obj in doc:
    print(obj)

# Extracting ComponentDefinitions from a Pre-existing Device

In [ ]:
# Extract the medium strength promoter `BBa_J23106` from your document.
medium_strength_promoter = doc.componentDefinitions['BBa_J23106']

# Extract the ribosomal binding site (rbs) `Q2` from your document.
rbs = doc.componentDefinitions['Q2']

# Extract the coding region (cds) `LuxR` from your document.
cds = doc.componentDefinitions['LuxR']

# Extract the terminator `ECK120010818` from your document.
terminator = doc.componentDefinitions['ECK120010818']

# Creating a New Device

In [ ]:
# Create a new empty device named `my_device`
my_device = doc.componentDefinitions.create('my_device')

# Assemble the new device from the promoter, rbs, cds, and terminator from above.
my_device.assemblePrimaryStructure(
    [medium_strength_promoter, rbs, cds, terminator])

# Set the role of the device with the Sequence Ontology term `gene`
my_device.roles = SO_GENE

# Compile the sequence for the new device
my_device.compile()

In [ ]:
for comp in my_device.getPrimaryStructure():
    print(comp.displayId)

# Managing a Design-Build-Test-Learn Workflow

In [ ]:
# Create a new design in your document called `my_design`. 
design = doc.designs.create('my_design')

# Set the structure of the design to `my_device` from above, and the function of the device to
# `None` (not covered in this tutorial)
design.structure = my_device
design.function = None

# Create three Activities [‘build`, `test`, `analysis`]
build_workflow_step = Activity('build')
test_workflow_step = Activity('test')
analysis_workflow_step = Activity('analysis')

# Create Plans for each Activity: set the`build` plan to `transformation`, the `test` plan
# to `promoter_characterization`, and the `analysis` plan to `parameter_optimization`
build_workflow_step.plan = Plan('transformation')
test_workflow_step.plan = Plan('promoter_characterization')
analysis_workflow_step.plan = Plan('parameter_optimization')

# Temporarily disable auto-construction of URIs (For setting Agent URIs)
setHomespace('')
Config.setOption('sbol_compliant_uris', False)

# Set Agents for each Activity: set the `build` agent to `mailto:jdoe@example.com`, the `test` agent
# to `http://sys-bio.org/plate_reader_1`, and the `analysis` agent to `http://tellurium.analogmachine.org`
build_workflow_step.agent = Agent('mailto:jdoe@example.com')
test_workflow_step.agent = Agent('http://sys-bio.org/plate_reader_1')
analysis_workflow_step.agent = Agent('http://tellurium.analogmachine.org')
# Re-enable auto-construction of URIs
setHomespace(namespace)
Config.setOption('sbol_compliant_uris', True)

# Add the three activities to your document.
doc.addActivity([build_workflow_step, test_workflow_step, analysis_workflow_step])

# Generate a build for your design out of your `build` activity. Name the result of the build step `transformed_cells`.
transformed_cells = build_workflow_step.generateBuild('transformed_cells', design)

# Generate a test for your build out of your `test` activity. Name the test `my_experiment`.
experiment = test_workflow_step.generateTest('my_experiement', transformed_cells)

# Generate an analysis of your test out of your `analysis` activity. Name the analysis `my_analysis`.
analysis = analysis_workflow_step.generateAnalysis('my_analysis', experiment)

In [ ]:
for obj in doc:
    print(obj)

# Uploading the Device Back to SynBioHub

In [ ]:
# Connect to your account on SynBioHub
import getpass
email = 'my@email.com'
password = getpass.getpass()
partshop.login(email, password)

# Give your document a displayId, name, and description
# (e.g. `my_device`, `my device`, `a newly characterized device`)
doc.displayId = 'awesome_device'
doc.name = "awesome device"
doc.description = "hey look, this is cool"

# Submit the document to the part shop
partshop.submit(doc)

In [ ]:
# Your document will now be available on your account at
# `https://synbiohub.org/user/<USERNAME>/<DOC.DISPLAYID>/<DOC.DISPLAYID>_collection`

# Attach the experimental results file `results.txt` to `my_experiment` from above
attachment_path = 'results.txt'
experiment_uri = 'https://synbiohub.org/user/my_username/awesome_device/Test_my_experiement/1'
partshop.attachFile(experiment_uri, attachment_path)